## Initiation aux modèles CNN avec la bibliothèque pytorch

Dans un premier temps nous allons construire une des premières architectures de CNN présenté par [Yann Le Cun](https://fr.wikipedia.org/wiki/Yann_Le_Cun), un [LeNet](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf).

L'architecture du LeNet est rappelée dans la figure ci-dessous:


![leNet5.png](leNet5.jpeg "Architecture Lenet")

In [1]:
# on importe les bibliothèques pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(42)

### Définition de la dataset [MNIST](https://fr.wikipedia.org/wiki/Base_de_donn%C3%A9es_MNIST) ainsi que ces transformations

In [2]:
from torchvision import datasets, transforms # On peut inporter directement la dataset de pytorch
from torch.utils.data import DataLoader 

# On définit transforms qui permet de redimensionner l'image en 32*32 et de la transformer en tensor
transforms = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5,), (0.5,))])

# On télécharge et on créer la dataset d'entraienement à l'aide du module datasets de torchvision
train_dataset = datasets.MNIST(root='mnist_data', 
                               train=True, 
                               transform=transforms,
                               download=True)

# On télécharge et on créer la dataset de test à l'aide du module datasets de torchvision
valid_dataset = datasets.MNIST(root='mnist_data', 
                               train=False, 
                               transform=transforms)

BATCH_SIZE = 32 #taille du batch size

# On définit le data loaders d'entraienement . Le data loaders permet de créer des batchs. On doit lui renseigner le batch size.
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)
# On définit le data loaders de validation . 
valid_loader = DataLoader(dataset=valid_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=False)

### Définition du modèle

In [3]:

class LeNet(nn.Module): # On créer la classe LeNet qui hérite de la classe mère Module

    def __init__(self): # On définit 
        super(LeNet, self).__init__()
        
        self.feature_extractor = nn.Sequential(  
            # Couche 1 (C1) : La première couche convolutive avec 6 noyaux de taille 5×5 et le stride de 1. 
            # Étant donné la taille de l'entrée (32×32×1), la sortie de cette couche est de taille 28×28×6.
            # kernel
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            
            #Couche 2 (S2) : Une couche de sous-échantillonnage/mise en commun avec 6 noyaux de taille 2×2.
            nn.AvgPool2d(kernel_size=2),
            
            
            #Couche 3 (C3) : La deuxième couche convolutive avec la même configuration que la première, cependant, 
            #cette fois avec 16 filtres. La sortie de cette couche est de 10×10×16.
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            
            #Couche 4 (S4) : La deuxième couche de mise en commun. La logique est identique à celle de la précédente, 
            #mais cette fois, la couche comporte 16 filtres. La sortie de cette couche est de taille 5×5×16.
            nn.AvgPool2d(kernel_size=2),
            
            #Couche 5 (C5) : La dernière couche convolutive avec 120 noyaux 5×5. 
            #Étant donné que l'entrée de cette couche est de taille 5×5×16 et que les noyaux sont de taille 5×5, 
            #la sortie est 1×1×120. Par conséquent, les couches S4 et C5 sont entièrement connectées. 
            #C'est aussi pourquoi dans certaines implémentations de LeNet-5, on utilise une couche entièrement connectée au lieu d'une couche convolutive comme 5ème couche. 
            #La raison pour laquelle cette couche reste une couche convolutive est le fait que si l'entrée du réseau est plus grande que celle utilisée dans l'entrée initiale (donc 32×32 dans ce cas), 
            #cette couche ne sera pas une couche entièrement connectée, car la sortie de chaque noyau ne sera pas 1×1.
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.BatchNorm2d(120),
            nn.ReLU()
        )
        
        ## La dernière couche est un réseau de neurones simple
        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.ReLU(),
            nn.Linear(in_features=84, out_features=10),
        )

    def forward(self, x): # on défini le passage de nos données
        
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits, probs
        

net = LeNet()
print(net) # On peut afficher les paramètres du modèle

LeNet(
  (feature_extractor): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (4): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (8): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (9): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): ReLU()
    (2): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [4]:
# On créer la fonction qui permet d'entrainer le modèle 

def train(train_loader, model, criterion, optimizer):
    '''
    Function for the training step of the training loop
    '''
    model.train() # on passe le modèle en entrainement
    running_loss = 0
    
    for X, y_true in train_loader:# on itére sur les données d'entrainement 

        optimizer.zero_grad() # on initialise l'erreur du gradient à zéro
        
        X = X.to(device) # on envoie les données X sur la GPU
        y_true = y_true.to(device) # on envoie les données Y sur la GPU
    
        # Forward pass (on passe les données dans le modèle)
        y_hat, _ = model(X) 
        loss = criterion(y_hat, y_true) # On calcul l'erreur du modèle avec la loss choisie
        # Rétropropagation du gradient
        loss.backward() 
        optimizer.step() # Descente de gradient (une itération)
        running_loss += loss.item()
        
    running_loss /= len(train_loader)
    return model, optimizer, running_loss

In [5]:
# On créer la fonction qui permet de valider le modèle 

def validate(valid_loader, model, criterion):
    '''
    Function for the validation step of the training loop
    '''
   
    model.eval()
    running_loss = 0
    
    for X, y_true in valid_loader:
    
        X = X.to(device)
        y_true = y_true.to(device)

        # Forward pass and record loss
        y_hat, _ = model(X) 
        loss = criterion(y_hat, y_true) 
        running_loss += loss.item()

    running_loss /= len(valid_loader)
    return model, running_loss

In [6]:
def training_loop(model, criterion, optimizer, train_loader, valid_loader, epochs, print_every=1):
    
    # Train model
    for epoch in range(0, epochs):
        train_loss = 0
        valid_loss = 0
        
        # training
        model, optimizer, train_loss = train(train_loader, model, criterion, optimizer)
    

        # validation
        with torch.no_grad(): # On désactive le gradient
            model, valid_loss = validate(valid_loader, model, criterion)
    

        if epoch % print_every == (print_every - 1):
            print("epoch",epoch)
            print("Train loss",train_loss)
            print("Valid loss",valid_loss)
    
    return model, optimizer

In [7]:
model = net.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [9]:
N_EPOCHS = 100
model, optimizer = training_loop(model, criterion, optimizer, train_loader, valid_loader, N_EPOCHS)

Train loss 0.16170943825278655
Valid loss 0.028729197495604472
Train loss 0.13781586179321514
Valid loss 0.02227735337645937
Train loss 0.129702695873864
Valid loss 0.02859245139701868
Train loss 0.12465118943516852
Valid loss 0.025408306968363006
Train loss 0.10804923721391094
Valid loss 0.028857562214756917
Train loss 0.09259195365094099
Valid loss 0.025678622803849638
Train loss 0.09941292782423987
Valid loss 0.034739617918578776
Train loss 0.08249669800973376
Valid loss 0.03446525464860366
Train loss 0.0811123874683862
Valid loss 0.027356936942945352
Train loss 0.07765725359103709
Valid loss 0.024225109543208376
Train loss 0.06980807149237736
Valid loss 0.02707069431498533
Train loss 0.06538203626365094
Valid loss 0.024094998203282033
Train loss 0.06122276970088275
Valid loss 0.026316775353564935
Train loss 0.06167250515005844
Valid loss 0.027720548768944607
Train loss 0.05706705474482516
Valid loss 0.02953696291071564
Train loss 0.06060498551991691
Valid loss 0.025077295111573673
